In [ ]:
import time
startScript = time.time()

### Imports and read in file

In [ ]:
import ROOT
import cmsstyle as CMS

file_TTnoPU = ROOT.TFile.Open("/eos/cms/store/user/etsai/VertexNtuples/TTToHadronic_noPU.root", "READ")
file_TTPU200 = ROOT.TFile.Open("/eos/cms/store/user/etsai/VertexNtuples/TTToHadronic_PU200.root", "READ")
dir_TTnoPU = file_TTnoPU.Get("vertexNtuplizer")
dir_TTPU200 = file_TTPU200.Get("vertexNtuplizer")
savePath = "/eos/home-b/btvweb/www/Offline/Phase2/etsai/Phase2/"
# dir_TTnoPU = file_TTnoPU.Get("vertexNtuplizerLoose")
# dir_TTPU200 = file_TTPU200.Get("vertexNtuplizerLoose")
# savePath = "/eos/home-b/btvweb/www/Offline/Phase2/etsai/Phase2/Loose/"

n = {}
p = {}

hexColors = ["#3f90da", "#ffa90e", "#bd1f01", "#94a4a2", "#832db6", "#a96b59", "#e76300", "#b9ac70", "#717581", "#92dadd"]
colors = [ROOT.Int_t(ROOT.TColor.GetColor(c)) for c in hexColors]

refHist = "nc"
nevents_TTnoPU = dir_TTnoPU.Get(refHist).GetEntries()
nevents_TTPU200 = dir_TTPU200.Get(refHist).GetEntries()
ratio = nevents_TTnoPU/nevents_TTPU200

print("There are %d TT no PU events and %d TT PU 200 events. The ratio is %.2f.\n"
        % (nevents_TTnoPU, nevents_TTPU200, ratio))

n_gv_dau_TTnoPU = dir_TTnoPU.Get("gv_trk_pt").GetEntries()
n_gv_dau_TTPU200 = dir_TTPU200.Get("gv_trk_pt").GetEntries() * ratio
n_gvs_dau_TTnoPU = dir_TTnoPU.Get("gvs_trk_pt").GetEntries()
n_gvs_dau_TTPU200 = dir_TTPU200.Get("gvs_trk_pt").GetEntries() * ratio
n_gvn_dau_TTnoPU = dir_TTnoPU.Get("gvn_trk_pt").GetEntries()
n_gvn_dau_TTPU200 = dir_TTPU200.Get("gvn_trk_pt").GetEntries() * ratio
n_gvns_dau_TTnoPU = dir_TTnoPU.Get("gvns_trk_pt").GetEntries()
n_gvns_dau_TTPU200 = dir_TTPU200.Get("gvns_trk_pt").GetEntries() * ratio
print("                 TTnoPU, TTPU200, noPU/PU200")
print("GV daughters:   %d, %d, %.4f" % (n_gv_dau_TTnoPU, n_gv_dau_TTPU200, n_gv_dau_TTnoPU/n_gv_dau_TTPU200))
print("GVs daughters:  %d, %d, %.4f" % (n_gvs_dau_TTnoPU, n_gvs_dau_TTPU200, n_gvs_dau_TTnoPU/n_gvs_dau_TTPU200))
print("GVn daughters:  %d, %d, %.4f" % (n_gvn_dau_TTnoPU, n_gvn_dau_TTPU200, n_gvn_dau_TTnoPU/n_gvn_dau_TTPU200))
print("GVns daughters: %d, %d, %.4f\n" % (n_gvns_dau_TTnoPU, n_gvns_dau_TTPU200, n_gvns_dau_TTnoPU/n_gvns_dau_TTPU200))

n_gv_TTnoPU = dir_TTnoPU.Get("gv_pt").GetEntries()
n_gv_TTPU200 = dir_TTPU200.Get("gv_pt").GetEntries() * ratio
n_gvs_TTnoPU = dir_TTnoPU.Get("gvs_pt").GetEntries()
n_gvs_TTPU200 = dir_TTPU200.Get("gvs_pt").GetEntries() * ratio
n_gvn_TTnoPU = dir_TTnoPU.Get("gvn_pt").GetEntries()
n_gvn_TTPU200 = dir_TTPU200.Get("gvn_pt").GetEntries() * ratio
n_gvns_TTnoPU = dir_TTnoPU.Get("gvns_pt").GetEntries()
n_gvns_TTPU200 = dir_TTPU200.Get("gvns_pt").GetEntries() * ratio
print("       TTnoPU, TTPU200, noPU/PU200")
print("GV:   %d, %d, %.4f" % (n_gv_TTnoPU, n_gv_TTPU200, n_gv_TTnoPU/n_gv_TTPU200))
print("GVs:   %d,  %d, %.4f" % (n_gvs_TTnoPU, n_gvs_TTPU200, n_gvs_TTnoPU/n_gvs_TTPU200))
print("GVn:  %d, %d, %.4f" % (n_gvn_TTnoPU, n_gvn_TTPU200, n_gvn_TTnoPU/n_gvn_TTPU200))
print("GVns:  %d,  %d, %.4f" % (n_gvns_TTnoPU, n_gvns_TTPU200, n_gvns_TTnoPU/n_gvns_TTPU200))
print(" GVs/GV:  %.4f, %.4f" % (n_gvs_TTnoPU/n_gv_TTnoPU, n_gvs_TTPU200/n_gv_TTPU200))
print(" GVn/GV:  %.4f, %.4f" % (n_gvn_TTnoPU/n_gv_TTnoPU, n_gvn_TTPU200/n_gv_TTPU200))
print("GVns/GVs: %.4f, %.4f" % (n_gvns_TTnoPU/n_gvs_TTnoPU, n_gvns_TTPU200/n_gvs_TTPU200))
print("GVns/GVn: %.4f, %.4f\n" % (n_gvns_TTnoPU/n_gvn_TTnoPU, n_gvns_TTPU200/n_gvn_TTPU200))

n_sv_trk_TTnoPU = dir_TTnoPU.Get("sv_trk_pt").GetEntries()
n_sv_trk_TTPU200 = dir_TTPU200.Get("sv_trk_pt").GetEntries() * ratio
n_svt_trk_TTnoPU = dir_TTnoPU.Get("svt_trk_pt").GetEntries()
n_svt_trk_TTPU200 = dir_TTPU200.Get("svt_trk_pt").GetEntries() * ratio
print("              TTnoPU,  TTPU200, noPU/PU200")
print("SV tracks:  %d, %d, %.2f" % (n_sv_trk_TTnoPU, n_sv_trk_TTPU200, n_sv_trk_TTnoPU/n_sv_trk_TTPU200))
print("SVt tracks: %d, %d, %.2f\n" % (n_svt_trk_TTnoPU, n_svt_trk_TTPU200, n_svt_trk_TTnoPU/n_svt_trk_TTPU200))

n_sv_TTnoPU = dir_TTnoPU.Get("sv_pt").GetEntries()
n_sv_TTPU200 = dir_TTPU200.Get("sv_pt").GetEntries() * ratio
n_svt_TTnoPU = dir_TTnoPU.Get("svt_pt").GetEntries()
n_svt_TTPU200 = dir_TTPU200.Get("svt_pt").GetEntries() * ratio
print("      TTnoPU,  TTPU200, noPU/PU200")
print("SV:  %d, %d, %.2f" % (n_sv_TTnoPU, n_sv_TTPU200, n_sv_TTnoPU/n_sv_TTPU200))
print("SVt: %d,  %d, %.2f" % (n_svt_TTnoPU, n_svt_TTPU200, n_svt_TTnoPU/n_svt_TTPU200))
print("SVt/SV: %.4f, %.4f\n" % (n_svt_TTnoPU/n_sv_TTnoPU, n_svt_TTPU200/n_sv_TTPU200))

na1 = dir_TTnoPU.Get("gv_sv_trk_pt").GetEntries()
na2 = dir_TTPU200.Get("gv_sv_trk_pt").GetEntries() * ratio
nb1 = dir_TTnoPU.Get("gv_svt_trk_pt").GetEntries()
nb2 = dir_TTPU200.Get("gv_svt_trk_pt").GetEntries() * ratio
nc1 = dir_TTnoPU.Get("gvs_sv_trk_pt").GetEntries()
nc2 = dir_TTPU200.Get("gvs_sv_trk_pt").GetEntries() * ratio
nd1 = dir_TTnoPU.Get("gvs_svt_trk_pt").GetEntries()
nd2 = dir_TTPU200.Get("gvs_svt_trk_pt").GetEntries() * ratio
ne1 = dir_TTnoPU.Get("gvn_sv_trk_pt").GetEntries()
ne2 = dir_TTPU200.Get("gvn_sv_trk_pt").GetEntries() * ratio
nf1 = dir_TTnoPU.Get("gvn_svt_trk_pt").GetEntries()
nf2 = dir_TTPU200.Get("gvn_svt_trk_pt").GetEntries() * ratio
ng1 = dir_TTnoPU.Get("gvns_sv_trk_pt").GetEntries()
ng2 = dir_TTPU200.Get("gvns_sv_trk_pt").GetEntries() * ratio
nh1 = dir_TTnoPU.Get("gvns_svt_trk_pt").GetEntries()
nh2 = dir_TTPU200.Get("gvns_svt_trk_pt").GetEntries() * ratio
print("(daughters) TTnoPU, TTPU200, noPU/PU200")
print("GV-SV:     %d, %d, %.2f" % (na1, na2, na1 / na2))
print("GV-SVt:    %d, %d, %.2f" % (nb1, nb2, nb1 / nb2))
print("GVs-SV:    %d, %d, %.2f" % (nc1, nc2, nc1 / nc2))
print("GVs-SVt:   %d, %d, %.2f" % (nd1, nd2, nd1 / nd2))
print("GVn-SV:    %d, %d, %.2f" % (ne1, ne2, ne1 / ne2))
print("GVn-SVt:   %d, %d, %.2f" % (nf1, nf2, nf1 / nf2))
print("GVns-SV:   %d, %d, %.2f" % (ng1, ng2, ng1 / ng2))
print("GVns-SVt:  %d, %d, %.2f" % (nh1, nh2, nh1 / nh2))
print("TTnoPU    GV-SVt / GV-SV   = %.3f" % (nb1 / na1))
print("TTnoPU   GVs-SVt / GVs-SV  = %.3f" % (nd1 / nc1))
print("TTnoPU   GVn-SVt / GVn-SV  = %.3f" % (nf1 / ne1))
print("TTnoPU  GVns-SVt / GVns-SV = %.3f" % (nh1 / ng1))
print("TTPU200   GV-SVt / GV-SV   = %.3f" % (nb2 / na2))
print("TTPU200  GVs-SVt / GVs-SV  = %.3f" % (nd2 / nc2))
print("TTPU200  GVn-SVt / GVn-SV  = %.3f" % (nf2 / ne2))
print("TTPU200 GVns-SVt / GVns-SV = %.3f\n" % (nh2 / ng2))

na1 = dir_TTnoPU.Get("gv_sv_pt").GetEntries()
na2 = dir_TTPU200.Get("gv_sv_pt").GetEntries() * ratio
nb1 = dir_TTnoPU.Get("gv_svt_pt").GetEntries()
nb2 = dir_TTPU200.Get("gv_svt_pt").GetEntries() * ratio
nc1 = dir_TTnoPU.Get("gvs_sv_pt").GetEntries()
nc2 = dir_TTPU200.Get("gvs_sv_pt").GetEntries() * ratio
nd1 = dir_TTnoPU.Get("gvs_svt_pt").GetEntries()
nd2 = dir_TTPU200.Get("gvs_svt_pt").GetEntries() * ratio
ne1 = dir_TTnoPU.Get("gvn_sv_pt").GetEntries()
ne2 = dir_TTPU200.Get("gvn_sv_pt").GetEntries() * ratio
nf1 = dir_TTnoPU.Get("gvn_svt_pt").GetEntries()
nf2 = dir_TTPU200.Get("gvn_svt_pt").GetEntries() * ratio
ng1 = dir_TTnoPU.Get("gvns_sv_pt").GetEntries()
ng2 = dir_TTPU200.Get("gvns_sv_pt").GetEntries() * ratio
nh1 = dir_TTnoPU.Get("gvns_svt_pt").GetEntries()
nh2 = dir_TTPU200.Get("gvns_svt_pt").GetEntries() * ratio
print("          TTnoPU, TTPU200, noPU/PU200")
print("GV-SV:    %d,  %d, %.2f" % (na1, na2, na1 / na2))
print("GV-SVt:   %d,  %d, %.2f" % (nb1, nb2, nb1 / nb2))
print("GVs-SV:   %d,  %d, %.2f" % (nc1, nc2, nc1 / nc2))
print("GVs-SVt:  %d,  %d, %.2f" % (nd1, nd2, nd1 / nd2))
print("GVn-SV:   %d,  %d, %.2f" % (ne1, ne2, ne1 / ne2))
print("GVn-SVt:  %d,  %d, %.2f" % (nf1, nf2, nf1 / nf2))
print("GVns-SV:  %d,  %d, %.2f" % (ng1, ng2, ng1 / ng2))
print("GVns-SVt: %d,  %d, %.2f" % (nh1, nh2, nh1 / nh2))
print("TTnoPU    GV-SVt / GV-SV   = %.3f" % (nb1 / na1))
print("TTnoPU   GVs-SVt / GVs-SV  = %.3f" % (nd1 / nc1))
print("TTnoPU   GVn-SVt / GVn-SV  = %.3f" % (nf1 / ne1))
print("TTnoPU  GVns-SVt / GVns-SV = %.3f" % (nh1 / ng1))
print("TTPU200   GV-SVt / GV-SV   = %.3f" % (nb2 / na2))
print("TTPU200  GVs-SVt / GVs-SV  = %.3f" % (nd2 / nc2))
print("TTPU200  GVn-SVt / GVn-SV  = %.3f" % (nf2 / ne2))
print("TTPU200 GVns-SVt / GVns-SV = %.3f" % (nh2 / ng2))

### Define plotting function for 1D histograms

In [ ]:
def plot(hists, labels, xlabel, ylabel, ylabelNorm, ylabelRatio, legHeader, log, saveName, plotRatio=0):
    CMS.cms_lumi = "Phase 2 Simulation"
    CMS.cms_energy = ""
    CMS.SetExtraText("Private work")

    ###################################### Not normalized ######################################

    left = hists[0].GetBinLowEdge(1)
    right = hists[0].GetBinLowEdge(hists[0].GetNbinsX() + 1)
    maxi = -1.0
    for hist in hists:
        maxi = max(maxi, hist.GetMaximum())

    canv = CMS.cmsCanvas(saveName,
        left, right, 0.8 if log else 0, 50 * maxi if log else 1.1 * maxi,
        xlabel, ylabel, square=CMS.kRectangular, extraSpace=0.01, iPos=0)
    canv.SetLogy(log)
    leg = CMS.cmsLeg(0.55, 0.89 - 0.04 * 8, 0.89, 0.89, textSize=0.04)
    if legHeader: CMS.cmsHeader(leg, legHeader, textSize=0.04)

    for iHist, hist in enumerate(hists):
        CMS.cmsDraw(hist, "hist", mcolor=colors[iHist], fstyle=0, lwidth=3)
        label = labels[iHist] + ", #mu=%.1f#pm%.1f" % (hist.GetMean(), hist.GetMeanError())
        leg.AddEntry(hist, label, "L")

    CMS.GetcmsCanvasHist(canv).GetXaxis().SetTitleOffset(1.2)
    CMS.GetcmsCanvasHist(canv).GetXaxis().SetLabelSize(0.04)
    CMS.GetcmsCanvasHist(canv).GetXaxis().SetTitleSize(0.05)
    CMS.GetcmsCanvasHist(canv).GetYaxis().SetTitleOffset(1.1)
    CMS.GetcmsCanvasHist(canv).GetYaxis().SetLabelSize(0.04)
    CMS.GetcmsCanvasHist(canv).GetYaxis().SetTitleSize(0.05)
    CMS.GetcmsCanvasHist(canv).GetYaxis().SetMaxDigits(1)
    ROOT.TGaxis.SetExponentOffset(-0.06, 0.01, "Y")

    CMS.SaveCanvas(canv, saveName + ".png", close=False)
    CMS.SaveCanvas(canv, saveName + ".pdf")

    ######################################### Normalized #########################################

    if plotRatio == 0: return

    normedHists = []
    ratioHists = []
    ratioColors = []
    ratioLabels = []
    for iHist, hist in enumerate(hists):
            normedHist = hist.Clone()
            normedHist.Scale(1 / hist.GetEntries())
            normedHists.append(normedHist)

    if plotRatio == 1:
        for iHist, hist in enumerate(normedHists):
            if iHist == 0: continue
            ratioHist = hist.Clone()
            ratioHist.Divide(normedHists[0])
            ratioHists.append(ratioHist)
            ratioLabels.append(labels[iHist] + "/" + labels[0])
            ratioColors.append(colors[iHist])

    if plotRatio == 2:
        if len(hists) != 4: print("You will probably get some errors!")
        ratioHist1 = normedHists[1].Clone()
        ratioHist1.Divide(normedHists[0])
        ratioHists.append(ratioHist1)
        ratioColors.append(colors[1])
        ratioLabels.append(labels[1] + "/" + labels[0])
        ratioHist2 = normedHists[3].Clone()
        ratioHist2.Divide(normedHists[2])
        ratioHists.append(ratioHist2)
        ratioColors.append(colors[3])
        ratioLabels.append(labels[3] + "/" + labels[2])

    maxiNorm = -1.0
    for hist in normedHists:
        maxiNorm = max(maxiNorm, hist.GetMaximum())
    maxiRatio = -1.0
    for hist in ratioHists:
        maxiRatio = max(maxiRatio, hist.GetMaximum())
    maxiRatio = max(2.0, maxiRatio)

    dicanv = CMS.cmsDiCanvas(saveName + "_ratio",
        left, right, 0, 1.1 * maxiNorm, 0.0, 1.1 * maxiRatio,
        xlabel, ylabelNorm, ylabelRatio, square=CMS.kRectangular, extraSpace=0.01, iPos=0)

    dicanv.cd(1)
    leg = CMS.cmsLeg(0.60, 0.89 - 0.05 * 5, 0.89, 0.89, textSize=0.05)
    if legHeader: CMS.cmsHeader(leg, legHeader, textSize=0.05)

    for iHist, hist in enumerate(normedHists):
        CMS.cmsDraw(hist, "hist", mcolor=colors[iHist], fstyle=0, lwidth=3)
        label = labels[iHist] + ", #mu=%.1f#pm%.1f" % (hist.GetMean(), hist.GetMeanError())
        leg.AddEntry(hist, label, "L")

    CMS.fixOverlay()

    dicanv.cd(2)
    leg_ratio = CMS.cmsLeg(0.37, 0.97 - 0.05 * 5, 0.85, 0.97, textSize=0.05, columns=4)

    for iHist, hist in enumerate(ratioHists):
        CMS.cmsDraw(hist, "hist", mcolor=ratioColors[iHist], fstyle=0, lwidth=3)
        leg_ratio.AddEntry(hist, ratioLabels[iHist], "PL")

    refLine = ROOT.TLine(left, 1, right, 1)
    CMS.cmsDrawLine(refLine, lcolor=ROOT.kBlack, lstyle=ROOT.kDotted)
    CMS.GetcmsCanvasHist(dicanv.cd(2)).GetXaxis().SetTitleOffset(1.2)
    CMS.GetcmsCanvasHist(dicanv.cd(2)).GetXaxis().SetTitleSize(0.125)

    CMS.SaveCanvas(dicanv, saveName + "_norm.png", close=False)
    CMS.SaveCanvas(dicanv, saveName + "_norm.pdf")

### Define plotting function for 2D histograms

In [ ]:
def Plot2D(hist, xlabel, ylabel, saveName):
    xmin = hist.GetXaxis().GetBinLowEdge(1)
    xmax = hist.GetXaxis().GetBinLowEdge(hist.GetNbinsX() + 1)
    ymin = hist.GetYaxis().GetBinLowEdge(1)
    ymax = hist.GetYaxis().GetBinLowEdge(hist.GetNbinsY() + 1)

    canv = CMS.cmsCanvas(saveName, xmin, xmax, ymin, ymax, xlabel, ylabel,
        square=CMS.kSquare, extraSpace=0.01, iPos=0, with_z_axis=True, scaleLumi=0.8)

    hist.GetZaxis().SetTitle("Events normalised")
    hist.GetZaxis().SetTitleOffset(1.4)
    hist.Draw("same colz")
    canv.SetLogz();

    CMS.SetAlternative2DColor(hist, CMS.cmsStyle) # Set a new palette
    CMS.UpdatePalettePosition(hist, canv) # Adjust palette position

    CMS.SaveCanvas(canv, saveName + ".png", close=False)
    CMS.SaveCanvas(canv, saveName + ".pdf")

### Define plotting function for efficiencies

In [ ]:
def plotEff(hists, labels, xlabel, ylabel, legHeader, saveName):
    CMS.cms_lumi = "Phase 2 Simulation"
    CMS.cms_energy = ""
    CMS.SetExtraText("Private work")

    left = hists[0].GetBinLowEdge(1)
    right = hists[0].GetBinLowEdge(hists[0].GetNbinsX() + 1)

    canv = CMS.cmsCanvas(saveName,
        left, right, 0.0, 1.2,
        xlabel, ylabel, square=CMS.kRectangular, extraSpace=0.01, iPos=0)
    leg = CMS.cmsLeg(0.55, 0.89 - 0.04 * 8, 0.89, 0.89, textSize=0.04)
    if legHeader: CMS.cmsHeader(leg, legHeader, textSize=0.04)

    for iHist, hist in enumerate(hists):
        CMS.cmsDraw(hist, "hist", mcolor=colors[iHist], fstyle=0, lwidth=3)
        leg.AddEntry(hist, labels[iHist], "L")

    CMS.GetcmsCanvasHist(canv).GetXaxis().SetTitleOffset(1.2)
    CMS.GetcmsCanvasHist(canv).GetXaxis().SetLabelSize(0.04)
    CMS.GetcmsCanvasHist(canv).GetXaxis().SetTitleSize(0.05)
    CMS.GetcmsCanvasHist(canv).GetYaxis().SetTitleOffset(1.1)
    CMS.GetcmsCanvasHist(canv).GetYaxis().SetLabelSize(0.04)
    CMS.GetcmsCanvasHist(canv).GetYaxis().SetTitleSize(0.05)

    CMS.SaveCanvas(canv, saveName + ".png", close=False)
    CMS.SaveCanvas(canv, saveName + ".pdf")

### Define histogram names and labels

In [ ]:
gv_names = ["gv", "gvs", "gvn", "gvns"]
gv_vars = ["trk_pt", "trk_pt2", "trk_eta", "trk_phi", "trk_charge",
        "x", "y", "z", "pt", "pt2", "eta", "phi", "motherPdgId", "pdgIdBin", "ntrk",
        "dxy", "dz", "d3d", "dxyerr", "dzerr", "d3derr", "dxysig", "dzsig", "d3dsig"]
gv_labels = ["daughter p_{T} [GeV]", "daughter p_{T}^{2} [GeV^{2}]", "daughter #eta", "daughter #phi", "daughter charge",
        "x [cm]", "y [cm]", "z [cm]", "p_{T} [GeV]", "p_{T}^{2} [GeV^{2}]", "#eta", "#phi", "mother PDG ID", "mother", "# of daughters",
        "d_{xy} [cm]", "d_{z} [cm]", "d_{3D} [cm]", "d_{xy} error [cm]", "d_{z} error [cm]", "d_{3D} error [cm]",
        "d_{xy} significance", "d_{z} significance", "d_{3D} significance"]

sv_names = ["sv", "svt"]
sv_vars = ["trk_tval", "trk_terr", "trk_tsig", "trk_tqual", "trk_pt", "trk_pt2", "trk_eta", "trk_phi",
        "trk_dxy", "trk_dz", "trk_d3d", "trk_dxyerr", "trk_dzerr", "trk_d3derr", "trk_dxysig", "trk_dzsig", "trk_d3dsig",
        "trk_charge", "trk_chi2", "trk_ndof", "trk_chi2dof",
        "x", "y", "z", "xerr", "yerr", "zerr",
        "dxy", "dz", "d3d", "dxyerr", "dzerr", "d3derr", "dxysig", "dzsig", "d3dsig",
        "pt", "pt2", "eta", "phi", "tavg", "trange", "chi2", "ndof", "chi2dof", "ntrk"]
sv_labels = ["track time [ns]", "track time error [ns]", "track time significance", "track time quality",
        "track p_{T} [GeV]", "track p_{T}^{2} [GeV^{2}]", "track #eta", "track #phi",
        "track d_{xy} [cm]", "track d_{z} [cm]", "track d_{3D} [cm]",
        "track d_{xy} error [cm]", "track d_{z} error [cm]", "track d_{3D} error [cm]",
        "track d_{xy} significance", "track d_{z} significance", "track d_{3D} significance",
        "track charge", "track #chi^{2}", "track N.D.O.F.", "track #chi^{2} / N.D.O.F.",
        "x [cm]", "y [cm]", "z [cm]", "x error [cm]", "y error [cm]", "z error [cm]",
        "d_{xy} [cm]", "d_{z} [cm]", "d_{3D} [cm]", "d_{xy} error [cm]", "d_{z} error [cm]", "d_{3D} error [cm]",
        "d_{xy} significance", "d_{z} significance", "d_{3D} significance",
        "p_{T} [GeV]", "p_{T}^{2} [GeV^{2}]", "#eta", "#phi", "<t> [ns]", "#Delta t [ns]",
        "#chi^{2}", "N.D.O.F.", "#chi^{2} / N.D.O.F.", "# of tracks"]

match_vars = ["xres", "yres", "zres", "xpull", "ypull", "zpull", "trk_deltaR", "trk_ptresnorm",
        "matchdxy", "matchd3d", "matchdxyerr", "matchd3derr", "matchdxysig", "matchd3dsig"]
match_labels = ["x resolution [cm]", "y resolution [cm]", "z resolution [cm]", "x pull", "y pull", "z pull",
        "SV track to GV daughter #Delta R", "|GV daughter p_{T} - SV track p_{T}| / (GV daughter p_{T} + SV track p_{T})",
        "SV to GV d_{xy} [cm]", "SV to GV d_{3D} [cm]", "SV to GV d_{xy} error [cm]", "SV to GV d_{3D} error [cm]",
        "SV to GV d_{xy} significance", "SV to GV d_{3D} significance"]
gv_sv_vars = gv_vars.copy()
gv_sv_vars.extend(match_vars)
gv_sv_labels = gv_labels.copy()
gv_sv_labels.extend(match_labels)
sv_gv_vars = sv_vars.copy()
sv_gv_vars.extend(match_vars)
sv_gv_labels = sv_labels.copy()
sv_gv_labels.extend(match_labels)

vars2D = ["matchdxy_dxy", "matchdxy_d3d", "matchd3d_dxy", "matchd3d_d3d", "trk_deltaR_ptresnorm"]
gv_labelsX = ["d_{xy} to SV", "d_{xy} to SV", "d_{3D} to SV", "d_{3D} to SV", "SV track to GV daughter #Delta R"]
gv_labelsY = ["d_{xy} to PV", "d_{3D} to PV", "d_{xy} to PV", "d_{3D} to PV", "|GV daughter p_{T} - SV track p_{T}| / (GV daughter p_{T} + SV track p_{T})"]
sv_labelsX = ["d_{xy} to GV", "d_{xy} to GV", "d_{3D} to GV", "d_{3D} to GV", "SV track to GV daughter #Delta R"]
sv_labelsY = gv_labelsY.copy()

sv_vars2D = ["eta_tavg", "eta_trange", "trk_eta_tval", "trk_eta_terr", "trk_eta_tsig", "trk_eta_tqual"]
sv_otherLabelsX = ["SV #eta", "SV #eta", "SV track #eta", "SV track #eta", "SV track #eta", "SV track #eta"]
sv_otherLabelsY = ["SV <t> [ns]", "SV #Delta t [ns]", "SV track time [ns]",
        "SV track time error [ns]", "SV track time significance", "SV track time quality"]

### Plot number of vertices and clusters

In [ ]:
n["ngv"] = dir_TTnoPU.Get("ngv").Clone()
p["ngv"] = dir_TTPU200.Get("ngv").Clone()
p["ngv"].Scale(ratio)
p["ngv"].SetEntries(p["ngv"].GetEntries() * ratio)
n["ngvs"] = dir_TTnoPU.Get("ngvs").Clone()
p["ngvs"] = dir_TTPU200.Get("ngvs").Clone()
p["ngvs"].Scale(ratio)
p["ngvs"].SetEntries(p["ngv"].GetEntries() * ratio)
n["ngvn"] = dir_TTnoPU.Get("ngvn").Clone()
p["ngvn"] = dir_TTPU200.Get("ngvn").Clone()
p["ngvn"].Scale(ratio)
p["ngvn"].SetEntries(p["ngv"].GetEntries() * ratio)
n["ngvns"] = dir_TTnoPU.Get("ngvns").Clone()
p["ngvns"] = dir_TTPU200.Get("ngvns").Clone()
p["ngvns"].Scale(ratio)
p["ngvns"].SetEntries(p["ngv"].GetEntries() * ratio)
n["nsv"] = dir_TTnoPU.Get("nsv").Clone()
p["nsv"] = dir_TTPU200.Get("nsv").Clone()
p["nsv"].Scale(ratio)
p["nsv"].SetEntries(p["ngv"].GetEntries() * ratio)
n["nsvt"] = dir_TTnoPU.Get("nsvt").Clone()
p["nsvt"] = dir_TTPU200.Get("nsvt").Clone()
p["nsvt"].Scale(ratio)
p["nsvt"].SetEntries(p["ngv"].GetEntries() * ratio)
n["nc"] = dir_TTnoPU.Get("nc").Clone()
p["nc"] = dir_TTPU200.Get("nc").Clone()
p["nc"].Scale(ratio)
p["nc"].SetEntries(p["ngv"].GetEntries() * ratio)
n["nct"] = dir_TTnoPU.Get("nct").Clone()
p["nct"] = dir_TTPU200.Get("nct").Clone()
p["nct"].Scale(ratio)
p["nct"].SetEntries(p["ngv"].GetEntries() * ratio)

start = time.time()

hists = [n["ngv"], n["ngvs"], n["ngvn"], n["ngvns"], p["ngv"], p["ngvs"], p["ngvn"], p["ngvns"]]
labels = ["t#bar{t} no PU, GV", "t#bar{t} no PU, GVs", "t#bar{t} no PU, GVn", "t#bar{t} no PU, GVns",
        "t#bar{t} PU 200, GV", "t#bar{t} PU 200, GVs", "t#bar{t} PU 200, GVn", "t#bar{t} PU 200, GVns"]
xlabel = "Number of gen vertices"
ylabel = "Events"
ylabelNorm = ""
legHeader = ""
saveName = savePath + "GenVertex/nGV"
plot(hists, labels, xlabel, ylabel, ylabelNorm, "", legHeader, False, saveName, 0)

hists = [n["nsv"], n["nsvt"], n["nc"], n["nct"], p["nsv"], p["nsvt"], p["nc"], p["nct"]]
labels = ["t#bar{t} no PU, SV", "t#bar{t} no PU, SVt", "t#bar{t} no PU, C", "t#bar{t} no PU, Ct",
        "t#bar{t} PU 200, SV", "t#bar{t} PU 200, SVt", "t#bar{t} PU 200, C", "t#bar{t} PU 200, Ct"]
xlabel = "Number of secondary vertices / IVF clusters"
ylabel = "Events"
ylabelNorm = ""
legHeader = ""
saveName = savePath + "SecondaryVertex/nSVnC"
plot(hists, labels, xlabel, ylabel, ylabelNorm, "", legHeader, False, saveName, 0)

end = time.time()
print("Plotting time was %.2f seconds." % (end - start))

### Plot GenVertex distributions

In [ ]:
for name in gv_names:
    for var in gv_vars:
        histName = name + "_" + var
        n[histName] = dir_TTnoPU.Get(histName).Clone()
        p[histName] = dir_TTPU200.Get(histName).Clone()
        p[histName].Scale(ratio)
        p[histName].SetEntries(p[histName].GetEntries() * ratio)


start = time.time()

labels = ["GV", "GVs", "GVn", "GVns"]
for iVar, var in enumerate(gv_vars):
    hists = [n["gv_" + var], n["gvs_" + var], n["gvn_" + var], n["gvns_" + var]]
    xlabel = "Gen vertex " + gv_labels[iVar]
    ylabel = "Events / %.2f" % n["gv_" + var].GetBinWidth(0)
    ylabelNorm = "Normalized events / %.2f" % n["gv_" + var].GetBinWidth(0)
    legHeader = "t#bar{t} no pileup"
    saveName = savePath + "GenVertex/TTnoPU_" + var
    plot(hists, labels, xlabel, ylabel, ylabelNorm, "Ratio", legHeader, False, saveName, 1)
for iVar, var in enumerate(gv_vars):
    hists = [p["gv_" + var], p["gvs_" + var], p["gvn_" + var], p["gvns_" + var]]
    xlabel = "Gen vertex " + gv_labels[iVar]
    ylabel = "Events / %.2f" % p["gv_" + var].GetBinWidth(0)
    ylabelNorm = "Normalized events / %.2f" % p["gv_" + var].GetBinWidth(0)
    legHeader = "t#bar{t} pileup 200"
    saveName = savePath + "GenVertex/TTPU200_" + var
    plot(hists, labels, xlabel, ylabel, ylabelNorm, "Ratio", legHeader, False, saveName, 1)

labels = ["GVn (noPU)", "GVns", "GVn (PU200)", "GVns"]
for iVar, var in enumerate(gv_vars):
    hists = [n["gvn_" + var], n["gvns_" + var], p["gvn_" + var], p["gvns_" + var]]
    xlabel = "Gen vertex " + gv_labels[iVar]
    ylabel = "Events / %.2f" % p["gvn_" + var].GetBinWidth(0)
    ylabelNorm = "Normalized events / %.2f" % p["gvn_" + var].GetBinWidth(0)
    legHeader = "GenVertex with no #nu"
    saveName = savePath + "GenVertex/" + var
    plot(hists, labels, xlabel, ylabel, ylabelNorm, "Ratio", legHeader, False, saveName, 1)

end = time.time()
print("Plotting time was %.2f seconds." % (end - start))

### Plot SecondaryVertex distributions

In [ ]:
for name in sv_names:
    for var in sv_vars:
        histName = name + "_" + var
        n[histName] = dir_TTnoPU.Get(histName).Clone()
        p[histName] = dir_TTPU200.Get(histName).Clone()
        p[histName].Scale(ratio)
        p[histName].SetEntries(p[histName].GetEntries() * ratio)

labels = ["SV (noPU)", "SVt", "SV (PU200)", "SVt"]

start = time.time()

for iVar, var in enumerate(sv_vars):
    hists = [n["sv_" + var], n["svt_" + var], p["sv_" + var], p["svt_" + var]]
    xlabel = "Secondary vertex " + sv_labels[iVar]
    ylabel = "Events / %.2f" % n["sv_" + var].GetBinWidth(0)
    ylabelNorm = "Normalized events / %.2f" % n["sv_" + var].GetBinWidth(0)
    legHeader = ""
    saveName = savePath + "SecondaryVertex/" + var
    plot(hists, labels, xlabel, ylabel, ylabelNorm, "Ratio", legHeader, False, saveName, 2)

end = time.time()
print("Plotting time was %.2f seconds." % (end - start))

### Plot matched distributions

1D GenVertex histograms

In [ ]:
for gv_name in gv_names:
    for sv_name in sv_names:
        for var in gv_sv_vars:
            histName = gv_name + "_" + sv_name + "_" + var
            n[histName] = dir_TTnoPU.Get(histName).Clone()
            p[histName] = dir_TTPU200.Get(histName).Clone()
            p[histName].Scale(ratio)
            p[histName].SetEntries(p[histName].GetEntries() * ratio)

start = time.time()

labels = ["GV (noPU)", "GVs", "GV (PU200)", "GVs"]
for sv_name in sv_names:
    for iVar, var in enumerate(gv_sv_vars):
        histName = "_" + sv_name + "_" + var
        hists = [n["gv" + histName], n["gvs" + histName], p["gv" + histName], p["gvs" + histName]]
        xlabel = "Gen vertex " + gv_sv_labels[iVar]
        ylabel = "Events / %.2f" % n["gv" + histName].GetBinWidth(0)
        ylabelNorm = "Normalized events / %.2f" % n["gv" + histName].GetBinWidth(0)
        legHeader = sv_name
        saveName = savePath + "GV-SV/gv_" + sv_name + "_" + var
        plot(hists, labels, xlabel, ylabel, ylabelNorm, "Ratio", legHeader, False, saveName, 2)

labels = ["GVn (noPU)", "GVns", "GVn (PU200)", "GVns"]
for sv_name in sv_names:
    for iVar, var in enumerate(gv_sv_vars):
        histName = "_" + sv_name + "_" + var
        hists = [n["gvn" + histName], n["gvns" + histName], p["gvn" + histName], p["gvns" + histName]]
        xlabel = "Gen vertex " + gv_sv_labels[iVar]
        ylabel = "Events / %.2f" % n["gvn" + histName].GetBinWidth(0)
        ylabelNorm = "Normalized events / %.2f" % n["gvn" + histName].GetBinWidth(0)
        legHeader = sv_name
        saveName = savePath + "GV-SV/gvn_" + sv_name + "_" + var
        plot(hists, labels, xlabel, ylabel, ylabelNorm, "Ratio", legHeader, False, saveName, 2)

end = time.time()
print("Plotting time was %.2f seconds." % (end - start))

2D GenVertex histograms

In [ ]:
start = time.time()

for gv_name in gv_names:
    for sv_name in sv_names:
        for iVar, var in enumerate(vars2D):
            histName = gv_name + "_" + sv_name + "_" + var

            n[histName] = dir_TTnoPU.Get(histName).Clone()
            saveName = savePath + "GV-SV/TTnoPU_" + histName
            Plot2D(n[histName], gv_labelsX[iVar], gv_labelsY[iVar], saveName)

            p[histName] = dir_TTPU200.Get(histName).Clone()
            p[histName].Scale(ratio)
            p[histName].SetEntries(p[histName].GetEntries() * ratio)
            saveName = savePath + "GV-SV/TTPU200_" + histName
            Plot2D(n[histName], gv_labelsX[iVar], gv_labelsY[iVar], saveName)

end = time.time()
print("Plotting time was %.2f seconds." % (end - start))

---

1D SecondaryVertex histograms

In [ ]:
for sv_name in sv_names:
    for gv_name in gv_names:
        for var in sv_gv_vars:
            histName = sv_name + "_" + gv_name + "_" + var
            n[histName] = dir_TTnoPU.Get(histName).Clone()
            p[histName] = dir_TTPU200.Get(histName).Clone()
            p[histName].Scale(ratio)
            p[histName].SetEntries(p[histName].GetEntries() * ratio)

labels = ["SV (noPU)", "SVt", "SV (PU200)", "SVt"]

start = time.time()

for gv_name in gv_names:
    for iVar, var in enumerate(sv_gv_vars):
        histName = "_" + gv_name + "_" + var
        hists = [n["sv" + histName], n["svt" + histName], p["sv" + histName], p["svt" + histName]]
        xlabel = "Secondary vertex " + sv_gv_labels[iVar]
        ylabel = "Events / %.2f" % n["sv" + histName].GetBinWidth(0)
        ylabelNorm = "Normalized events / %.2f" % n["sv" + histName].GetBinWidth(0)
        legHeader = gv_name
        saveName = savePath + "SV-GV/sv_" + gv_name + "_" + var
        plot(hists, labels, xlabel, ylabel, ylabelNorm, "Ratio", legHeader, False, saveName, 2)

end = time.time()
print("Plotting time was %.2f seconds." % (end - start))

2D SecondaryVertex histograms

In [ ]:
start = time.time()

for sv_name in sv_names:
    for gv_name in gv_names:
        for iVar, var in enumerate(vars2D):
            histName = sv_name + "_" + gv_name + "_" + var

            n[histName] = dir_TTnoPU.Get(histName).Clone()
            saveName = savePath + "SV-GV/TTnoPU_" + histName
            Plot2D(n[histName], sv_labelsX[iVar], sv_labelsY[iVar], saveName)

            p[histName] = dir_TTPU200.Get(histName).Clone()
            p[histName].Scale(ratio)
            p[histName].SetEntries(p[histName].GetEntries() * ratio)
            saveName = savePath + "SV-GV/TTPU200_" + histName
            Plot2D(n[histName], sv_labelsX[iVar], sv_labelsY[iVar], saveName)

        for iVar, var in enumerate(sv_vars2D):
            histName = sv_name + "_" + gv_name + "_" + var

            n[histName] = dir_TTnoPU.Get(histName).Clone()
            saveName = savePath + "SV-GV/TTnoPU_" + histName
            Plot2D(n[histName], sv_otherLabelsX[iVar], sv_otherLabelsY[iVar], saveName)

            p[histName] = dir_TTPU200.Get(histName).Clone()
            p[histName].Scale(ratio)
            p[histName].SetEntries(p[histName].GetEntries() * ratio)
            saveName = savePath + "SV-GV/TTPU200_" + histName
            Plot2D(n[histName], sv_otherLabelsX[iVar], sv_otherLabelsY[iVar], saveName)

end = time.time()
print("Plotting time was %.2f seconds." % (end - start))

### Calculate and plot efficiencies

In [ ]:
for gv_name in gv_names:
    for sv_name in sv_names:
        for var in gv_vars:
            nameNum = gv_name + "_" + sv_name + "_" + var
            nameDen = gv_name + "_" + var
            nameEff = gv_name + "_" + sv_name + "_" + var + "_eff"
            n[nameEff] = dir_TTnoPU.Get(nameNum).Clone()
            n[nameEff].Divide(n[nameNum], n[nameDen], 1.0, 1.0, option="B")
            p[nameEff] = dir_TTPU200.Get(nameNum).Clone()
            p[nameEff].Divide(p[nameNum], p[nameDen], 1.0, 1.0, option="B")

start = time.time()

labels = ["GV (noPU)", "GVs", "GV (PU200)", "GVs"]
for sv_name in sv_names:
    for iVar, var in enumerate(gv_vars):
        histName = "_" + sv_name + "_" + var + "_eff"
        hists = [n["gv" + histName], n["gvs" + histName], p["gv" + histName], p["gvs" + histName]]
        xlabel = "Gen vertex " + gv_sv_labels[iVar]
        ylabel = "Matching efficiency / %.2f" % n["gv" + histName].GetBinWidth(0)
        legHeader = sv_name
        saveName = savePath + "Efficiency/gv_" + sv_name + "_" + var + "_eff"
        plotEff(hists, labels, xlabel, ylabel, legHeader, saveName)

labels = ["GVn (noPU)", "GVns", "GVn (PU200)", "GVns"]
for sv_name in sv_names:
    for iVar, var in enumerate(gv_vars):
        histName = "_" + sv_name + "_" + var + "_eff"
        hists = [n["gvn" + histName], n["gvns" + histName], p["gvn" + histName], p["gvns" + histName]]
        xlabel = "Gen vertex " + gv_sv_labels[iVar]
        ylabel = "Matching efficiency / %.2f" % n["gvn" + histName].GetBinWidth(0)
        legHeader = sv_name
        saveName = savePath + "Efficiency/gvn_" + sv_name + "_" + var + "_eff"
        plotEff(hists, labels, xlabel, ylabel, legHeader, saveName)

end = time.time()
print("Plotting time was %.2f seconds." % (end - start))

In [ ]:
endScript = time.time()
print("Runtime was %.2f minutes." % ((endScript - startScript) / 60.0))

### Plotting script done! Check https://btvweb.web.cern.ch/Offline/Phase2/etsai/ for plots.